# Generate a template of the weighted average of two samples.

In [7]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def model_uncern(covariance_matrix,x):
    var_a = covariance_matrix[0][0]
    var_b = covariance_matrix[1][1]
    var_c = covariance_matrix[2][2]
    
    cov_ab = covariance_matrix[0][1]
    cov_ac = covariance_matrix[0][2]
    cov_bc = covariance_matrix[1][2]
    
    diagonal_term = var_a*(x**4) + var_b*(x**2) + var_c
    no_diagonal_term = 2*cov_ab*(x**3) + 2*cov_ac*(x**2) + 2*cov_bc*x
    
    return np.sqrt(diagonal_term+no_diagonal_term)

def parabolic_shape(mjj,parameters):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    return a*mjj*mjj+b*mjj+c

model_uncer = np.vectorize(model_uncern)
parabolic_shape = np.vectorize(parabolic_shape,excluded=['parameters'])

fitParams = {"Sherpa": [ 1.31867345e-07, -6.91194368e-04,  1.49568233e+00], 
              "MG" : [ 1.35194973e-07, -5.44814667e-04,  9.89623841e-01] }

covarianceMatrix =  {"Sherpa" : [[ 1.19177067e-17, -2.94876918e-14,  1.41446066e-11],
                               [-2.94876918e-14,  8.14817316e-11, -4.30728335e-08],
                               [ 1.41446066e-11, -4.30728335e-08,  2.66545658e-05]],
                    "MG" : [[ 6.10103774e-18, -1.39152060e-14,  6.28353598e-12],
                           [-1.39152060e-14,  3.55524563e-11, -1.77055103e-08],
                           [ 6.28353598e-12, -1.77055103e-08,  1.02381778e-05]]}

def scaleBinUncertainty(histogram,sampleName):
    
    sampleType = "MG"
    if "Sherpa" in sampleName:
        sampleType = "Sherpa"
    
    for i in range(1,histogram.GetNbinsX()+1):
        x = histogram.GetBinCenter(i)
        error = histogram.GetBinError(i)
        rw = parabolic_shape(x,parameters=fitParams[sampleType])
        rw_error = model_uncern(covarianceMatrix[sampleType],x)
        newError = np.sqrt(error**2 + ((rw_error/rw)**2)*(error**2))
        histogram.SetBinError(i,newError)

import sys
sys.path.append('/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/')


############# CONFIGURING THE JOB ###########################
samples_path = "/Users/diegomac/Documents/HEP/VBF-Analysis/Zll/High-Mass/SR/"
sample1_name = "Zll_SherpaRW"
sample2_name = "Zll_MGRW"

qcd_1=0.923
qcd_2=0.968

fs_name = "Zll"
EWSampleUsedForExtraction = "PoPy"
EWSampleUsedForExtraction = "_EW-"+EWSampleUsedForExtraction


In [8]:
from histogramHelpers import histogramsHighStatsZtautau,histogramsLowStatsZtautau,histogramsHighStatsZll,histogramsLowStatsZll
from histogramHelpers import biner,normalization

# Define what channel histograms to produce
histos = histogramsLowStatsZll

r.TH1.AddDirectory(r.kFALSE)
file1 = r.TFile.Open(samples_path+sample1_name+'.root')
file2 = r.TFile.Open(samples_path+sample2_name+'.root')

myFile =r.TFile.Open(fs_name+"_Average"+EWSampleUsedForExtraction+".root", "RECREATE")
r.TH1.AddDirectory(r.kFALSE)    

for histo,value in histos.items():
    print(histo)
    h1 = file1.Get(histo)
    h2 = file2.Get(histo)
    
    h1.Scale(qcd_1)
    h2.Scale(qcd_2)
    
    if len(value)>2:
    
        rebining=biner(histos[histo][0],histos[histo][1],h1)
        nb=len(rebining)-1
        h1=h1.Rebin(nb,histo,rebining)
        h2=h2.Rebin(nb,histo,rebining)
        
        if "mass_jj" in histo:
            scaleBinUncertainty(h1,sample1_name)
            scaleBinUncertainty(h2,sample2_name)
        
        #h1.Scale(1.0/h1.Integral(1,-1))
        #h2.Scale(1.0/h2.Integral(1,-1))

        #hist_list=[h1,h2]
        #normalization(hist_list,total_histos[histo][2])
    
    h1.SetBit(r.TH1.kIsAverage)
    h2.SetBit(r.TH1.kIsAverage)
    
    final_hist = h1.Clone()
    final_hist.Add(h2)

    myFile.WriteObject(final_hist,histo)

myFile.Close()

os.system("mv "+fs_name+"_Average"+EWSampleUsedForExtraction+".root "+samples_path+"/"+fs_name+"_Average"+EWSampleUsedForExtraction+".root")

n_bjets
lepiso
n_jets_interval
lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
ljet0_eta_basic_cuts_ptl
ljet1_eta_basic_cuts_ptl
delta_R_leplep_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
delta_R_lep1jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
delta_R_lep2jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
delta_phi
lep1_pt
lep2_pt
ljet0_pt
ljet1_pt
pt_bal
Z_centrality
delta_y
inv_mass
mass_jj
Z_pt_reco_basic_cuts_ptl
vec_sum_pt_jets_basic_cuts_ptl
ratio_zpt_sumjetpt_basic_cuts_ptl
met_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl


0